In [1]:
# import numpy as np
# from lines.Line import Line
# from prototypes.models.PrototypeClassifierModels import CLASSIFIER_MODEL_2NN
# from training.CrossEntropyTraining import ClassifierTrainerWithCrossEntropyNoGen
# from datautils.LEOPARDEncoderUtils import get_labelled_LEOPARD_training_data as get_labelled_training_data
# from lines.LineGenerator import LineGenerator
# # from evaluation.Evaluator import Evaluator

# test_params = {
#   'type': 'cross_entropy',
#   'encoder': "bert",
# }

# trainer_params = {
#   'shuffle': True,
#   'num_workers': 0
# }

# batch_size = {
#   4: 4, 8: 8, 16: 12
# }

# epochs = {
#   4: {
#     2: 15, 3: 30, 4: 45, 5: 60, 6: 75
#   },
#   8: {
#     2: 30, 3: 40, 4: 50, 5: 70, 6: 100
#   },
#   16: {
#     2: 15, 3: 25, 4: 35, 5: 45, 6: 55
#   },
# }

# learning_rates = {
#   4: 5e-3,
#   8: 1e-2,
#   16: 5e-2
# }

# training_params = {
#   'encoder': "bert",
#   'epochs': epochs,
#   'warmupSteps': 100,
#   'reduction': 'none',
#   'learning_rate': learning_rates,
#   'printValidationPlot': True,
#   'printValidationLoss': False
# }

# for category in ["rating_dvd"]:
#     training_params['category'] = category
#     for episode in range(10):
#         training_params['episode'] = episode
#         for shot in [4]:
#             training_params['shot'] = shot
#             trainer_params['batch_size'] = batch_size[shot]
#             training_encodings, training_labels, label_keys = get_labelled_training_data(category, shot, episode)
            
#             if training_encodings.shape[0] == 0:
#                 continue
            
#             training_set = {}
#             training_set['encodings'] = training_encodings
#             training_set['labels'] = training_labels
            
#             lineGenerator = LineGenerator(training_set, CLASSIFIER_MODEL_2NN)
#             classifierTrainer = ClassifierTrainerWithCrossEntropyNoGen(lineGenerator, trainer_params, training_params, label_keys)
#             classifierTrainer.trainPrototypes(training_params, trainer_params, training_set)
#             lines = classifierTrainer.getLines()
            
# #             evaluator = Evaluator(label_keys, test_params)
# #             evaluator.evaluate(category, shot, episode)

In [2]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
print(cola)
sst2 = load_dataset('glue','sst2')
print(sst2)
pt_cola = GLUEDataset([cola, sst2], 'train')
print(pt_cola)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1044.57it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})


Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1228.56it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [3]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=2, nWay=4, batchSize=16, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(5):
    data, targets = next(iter(train_protomaml_loader))
    print(data)
    print(targets)



ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/aksingh/Documents/meta-learned-lines/samplers/FewShotEpisodeBatchSampler.py", line 30, in collate
    data, labels = dataset[idx * self.batchSize: (idx + 1) * self.batchSize]
ValueError: too many values to unpack (expected 2)


In [ ]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [ ]:
from training.models.FOMAML import FOMAML

params = {"batch_size":1}
fomaml = FOMAML(params=params, labelKeys=None)
fomaml.trainPrototypes(gm_ds, params)

In [ ]:
train_set = pt_cola
train_protomaml_sampler = 
train_protomaml_loader = data.DataLoader(
    train_set, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.get_collate_fn(), num_workers=2
)